In [42]:
# Step 1: 환경 변수 및 패키지 로드
import os
import pickle
import json

from openai import AsyncOpenAI
from pymilvus import connections, CollectionSchema, FieldSchema, DataType, Collection
from dotenv import load_dotenv
import openai
from openai.types import CreateEmbeddingResponse, Embedding


def load_environment_variables():
    load_dotenv()
    openai.api_key = os.getenv("OPEN_API_KEY")

load_environment_variables()


In [43]:
# Step 2: Milvus 연결 및 데이터 로드
def connect_to_milvus():
    connections.connect(alias="default", host="localhost", port="19530")

def load_data(file_path):
    with open(file_path, 'rb') as file:
        return pickle.load(file)

connect_to_milvus()
raw_data = load_data('../final_result.pkl')


/Users/yuhyeongchan/Library/Caches/pypoetry/virtualenvs/faq-chatbot-6f8XOInD-py3.12/lib/python3.12/site-packages/grpc/_interceptor.py:688: RuntimeWarning: coroutine 'generate_faq_embeddings' was never awaited
  def unary_unary(


In [44]:
# Step 3: 데이터 처리 및 저장
def process_data(raw_data):
    return [{"question": k, "answer": v} for k, v in raw_data.items()]

def save_to_json(data, file_path):
    with open(file_path, 'w') as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)

faqs = process_data(raw_data)
save_to_json(faqs, 'faqs.json')

print(f"Sample FAQ: {faqs[0]}")


Sample FAQ: {'question': '[가입절차] 스마트스토어센터 회원가입은 어떻게 하나요? (ID만들기)', 'answer': '네이버 커머스 ID 하나로 스마트스토어센터와 같은 네이버의 다양한 커머스 서비스를 편리하게 이용하실 수 있습니다.네이버 커머스 ID가 없으시다면 [가입하기] 버튼을 통해 회원가입해 주세요.\xa0\xa0 \xa0 \xa0 \xa0 \xa0\xa01. \'네이버ID\' 로 네이버 커머스 ID 가입하기1) [네이버 아이디로 가입하기] 버튼을 눌러서 네이버 로그인 해 주세요.2) 연락가능한 휴대폰번호를 입력하시고 인증하신 후 개인정보 수집동의를 체크 하여 [가입] 버튼을 눌러주세요.\xa03) 네이버 커머스 ID 회원가입이 완료되어 해당 네이버ID 로 스마트스토어센터 서비스 가입을 하실 수 있습니다.\xa0이 후 스마트스토어센터 로그인 시에 [네이버 아이디로 로그인] 버튼을 눌러서 네이버 로그인을 하실 수 있습니다.\xa02. \'이메일 아이디\'로 네이버 커머스 ID 가입하기\xa01) [이메일 아이디로 가입하기] 버튼을 눌러서 사용할 ID를 입력해 주세요.! 중요. 로그인 ID는 "실 사용중인 이메일주소"로 기재하셔야 합니다. (예 : abc@naver.com)2) 연락가능한 휴대폰번호와 이메일주소를 입력하시고 인증하신 후 개인정보 수집동의를 체크 하여 [가입] 버튼을 눌러주세요.3) 네이버 커머스 ID 회원가입이 완료되어 해당 이메일 아이디로 스마트스토어센터 서비스 가입을 하실 수 있습니다.이 후 스마트스토어센터 로그인 시에 해당 이메일 아이디와 비밀번호를 입력하여 로그인을 하실 수 있습니다.\xa0※ 참고. 로그인 ID 계정 대표자와 사업자번호 대표자 동일여부는 체크하지 않습니다. (대표자가 상이해도 가입 진행 가능합니다.)\xa0 \xa0\xa0 \xa0 \xa0 \xa0 \xa0\xa0\n\n\n위 도움말이 도움이 되었나요?\n\n\n별점1점\n\n별점2점\n\n별점3점\n\n별점4점\n\n별점5점\n\n\n\n소중한

In [67]:
# Step 4: Milvus 컬렉션 정의 및 생성
def define_collection_schema():
    fields = [
        FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name="faq_index", dtype=DataType.INT64),  # 해당 chunk가 어느 FAQ인지 식별
        FieldSchema(name="chunk_index", dtype=DataType.INT64),  # FAQ 내 chunk 순서
        FieldSchema(name="question", dtype=DataType.VARCHAR, max_length=1024),
        FieldSchema(name="answer", dtype=DataType.VARCHAR, max_length=30000),
        FieldSchema(name="chunk_text", dtype=DataType.VARCHAR, max_length=2048),
        FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=1536),
    ]
    return CollectionSchema(fields, description="FAQ collection with chunks")

def create_collection(schema):
    collection = Collection("faq_collection", schema)
    index_params = {
        "index_type": "IVF_FLAT",
        "metric_type": "IP",
        "params": {"nlist": 128}
    }
    collection.create_index(field_name="embedding", index_params=index_params)
    return collection

schema = define_collection_schema()
collection = create_collection(schema)


In [59]:
def generate_embeddings(questions):
    embeddings = []
    for question in questions:
        response: CreateEmbeddingResponse = openai.embeddings.create(
            model="text-embedding-3-small",
            input=question
        )
        embedding = response.data
        embeddings.append(embedding)
    return embeddings


In [65]:
import asyncio
import openai
def sliding_window_split(text, chunk_size=200, overlap=50):
    # overlap은 chunk 사이에 겹칠 문자 수
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        if end >= len(text):
            break
        start = end - overlap
    return chunks

def embed_text(texts, model="text-embedding-3-small"):
    # texts: 리스트 형태의 문자열
    # openai.Embedding.acreate를 통해 비동기 임베딩 요청
    tasks = []
    reses = []
    for t in texts:
        tasks.append(openai.embeddings.create(model=model, input=t))
        res : CreateEmbeddingResponse = openai.embeddings.create(model=model, input=t)
        reses.append(res)
    responses = reses
    # responses: 각 response는 openai response 객체
    # embeddings: [(텍스트, 임베딩 벡터), ...]
    embeddings = []
    for text, resp in zip(texts, responses):
        # resp.data[0].embedding 형태로 임베딩 벡터 접근 가능
        embedding_vector = resp.data[0].embedding
        embeddings.append((text, embedding_vector))
    return embeddings

def generate_faq_embeddings(faqs, chunk_size=500, overlap=100):
    all_texts = []
    faq_metadata = []  # (faq_index, chunk_index)에 대한 메타데이터 저장

    for i, faq in enumerate(faqs):
        combined_text = f"Q: {faq['question']}\nA: {faq['answer']}"
        chunks = sliding_window_split(combined_text, chunk_size=chunk_size, overlap=overlap)

        for j, chunk in enumerate(chunks):
            all_texts.append(chunk)
            faq_metadata.append((i, j))

    embedded_results = embed_text(all_texts)

    final_data = []
    for (chunk_text, embedding_vector), (faq_idx, chunk_idx) in zip(embedded_results, faq_metadata):
        final_data.append({
            "faq_index": faq_idx,
            "chunk_index": chunk_idx,
            "embedding": embedding_vector,
            "chunk_text": chunk_text,
            "question": faqs[faq_idx]["question"],
            "answer": faqs[faq_idx]["answer"]
        })

    return final_data

In [66]:
result = generate_faq_embeddings(faqs)
# 데이터 삽입
collection.insert(result)

RPC error: [insert_rows], <MilvusException: (code=1100, message=length of varchar field answer exceeds max length, row number: 3929, length: 22160, max length: 20000: invalid parameter)>, <Time:{'RPC start': '2024-12-08 00:27:40.684034', 'RPC error': '2024-12-08 00:27:42.211386'}>


MilvusException: <MilvusException: (code=1100, message=length of varchar field answer exceeds max length, row number: 3929, length: 22160, max length: 20000: invalid parameter)>

In [68]:
collection.insert(result)


(insert count: 4371, delete count: 0, upsert count: 0, timestamp: 454449009652662274, success count: 4371, err count: 0

In [ ]:
# Step 6: 컬렉션 정보 출력
def print_collection_info(collection):
    print("컬렉션 필드 정보:")
    for field in collection.schema.fields:
        print(f"필드 이름: {field.name}, 데이터 타입: {field.dtype}")

print_collection_info(collection)


In [ ]:
# Step 7: 컬렉션 쿼리 및 결과 출력
def query_collection(collection):
    collection.load()
    search_params = {"metric_type": "IP", "params": {"nprobe": 10}}
    query_vector = collection.query(
        expr="faq_id >= 0",
        output_fields=["embedding", "faq"],
        limit=5
    )
    return query_vector

def print_query_results(query_vector):
    for result in query_vector:
        print(f"임베딩 데이터: {result['embedding']}")
        print(f"FAQ 데이터: {result['faq']}")

query_vector = query_collection(collection)
print_query_results(query_vector)


In [60]:


# Step 7: 컬렉션 쿼리 및 결과 출력
q1 = "네이버 쇼핑윈도 노출 절차는 어떻게 되나요?"
data = generate_embeddings([q1])[0][0].embedding


In [63]:
print(len(data))
search_vector = [data]
for float_data in search_vector:
    print(type(float_data))
print(type(search_vector))  # 데이터가 리스트인지 확인
print(len(search_vector))   # 리스트 길이 확인
search_param = {
    "metric_type": "IP",  # 유사도 측정 방식
    "params": {"nprobe": 10}  # 검색 정확도 조정
}

results = collection.search(
    data=search_vector,
    anns_field="embedding",  
    param=search_param,
    limit=5, 
    output_fields=["answer"]  # 반환할 추가 필드
)

# 결과 출력
for i, result in enumerate(results[0]):
    print(f"Rank {i + 1}: id={result.id}, distance={result.distance}, faq={result.entity.get('answer')}")

1536
<class 'list'>
<class 'list'>
1
Rank 1: id=454437888679314291, distance=0.36946576833724976, faq=네이버 커머스 ID 하나로 스마트스토어센터와 같은 네이버의 다양한 커머스 서비스를 편리하게 이용하실 수 있습니다.네이버 커머스 ID가 없으시다면 [가입하기] 버튼을 통해 회원가입해 주세요.           1. '네이버ID' 로 네이버 커머스 ID 가입하기1) [네이버 아이디로 가입하기] 버튼을 눌러서 네이버 로그인 해 주세요.2) 연락가능한 휴대폰번호를 입력하시고 인증하신 후 개인정보 수집동의를 체크 하여 [가입] 버튼을 눌러주세요. 3) 네이버 커머스 ID 회원가입이 완료되어 해당 네이버ID 로 스마트스토어센터 서비스 가입을 하실 수 있습니다. 이 후 스마트스토어센터 로그인 시에 [네이버 아이디로 로그인] 버튼을 눌러서 네이버 로그인을 하실 수 있습니다. 2. '이메일 아이디'로 네이버 커머스 ID 가입하기 1) [이메일 아이디로 가입하기] 버튼을 눌러서 사용할 ID를 입력해 주세요.! 중요. 로그인 ID는 "실 사용중인 이메일주소"로 기재하셔야 합니다. (예 : abc@naver.com)2) 연락가능한 휴대폰번호와 이메일주소를 입력하시고 인증하신 후 개인정보 수집동의를 체크 하여 [가입] 버튼을 눌러주세요.3) 네이버 커머스 ID 회원가입이 완료되어 해당 이메일 아이디로 스마트스토어센터 서비스 가입을 하실 수 있습니다.이 후 스마트스토어센터 로그인 시에 해당 이메일 아이디와 비밀번호를 입력하여 로그인을 하실 수 있습니다. ※ 참고. 로그인 ID 계정 대표자와 사업자번호 대표자 동일여부는 체크하지 않습니다. (대표자가 상이해도 가입 진행 가능합니다.)             


위 도움말이 도움이 되었나요?


별점1점

별점2점

별점3점

별점4점

별점5점



소중한 의견을 남겨주시면 보완하도록 노력하겠습니다.

보내기



관련 도움말/키워드

스마트스토어 로그인ID

In [58]:
## show token usage
import requests
import datetime

# OpenAI API Key
api_key = os.getenv("OPEN_API_KEY")

# 현재 날짜 및 시간
end_date = datetime.datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.datetime.now() - datetime.timedelta(days=3)).strftime('%Y-%m-%d')  #

# Usage API 엔드포인트
url = f"https://api.openai.com/v1/usage?date={datetime.datetime.now().strftime('%Y-%m-%d')}"

# API 요청
response = requests.get(
    url,
    headers={
        "Authorization": f"Bearer {api_key}"
    }
)

# 결과 확인
if response.status_code == 200:
    usage_data = response.json()
    print("사용량 데이터:", usage_data)
else:
    print(f"API 요청 실패: {response.status_code}, {response.text}")


사용량 데이터: {'object': 'list', 'data': [{'organization_id': 'org-dfQXmNQPbPJvHupLOT9HmChV', 'organization_name': 'Personal', 'aggregation_timestamp': 1733538480, 'n_requests': 1, 'operation': 'embeddings', 'snapshot_id': 'text-embedding-ada-002-v2', 'n_context_tokens_total': 10, 'n_generated_tokens_total': 0, 'email': None, 'api_key_id': None, 'api_key_name': None, 'api_key_redacted': None, 'api_key_type': None, 'project_id': None, 'project_name': None, 'request_type': '', 'n_cached_context_tokens_total': 0}, {'organization_id': 'org-dfQXmNQPbPJvHupLOT9HmChV', 'organization_name': 'Personal', 'aggregation_timestamp': 1733538600, 'n_requests': 1, 'operation': 'embeddings', 'snapshot_id': 'text-embedding-ada-002-v2', 'n_context_tokens_total': 28, 'n_generated_tokens_total': 0, 'email': None, 'api_key_id': None, 'api_key_name': None, 'api_key_redacted': None, 'api_key_type': None, 'project_id': None, 'project_name': None, 'request_type': '', 'n_cached_context_tokens_total': 0}, {'organizatio